GitHub: https://github.com/vidisha04/02467-CSS.git
Group Members:
* Vidisha Sinha: s204081
* Mads Andersen: s204137

# Part 1 - Webscraping the 2019 poster and oral presentations
Now we hav to scrape the two sessions both the oral and poster, at the same time the following questions will be answered:
1) Inspect the HTML of the pages above and use web-scraping to get the set of participants in 2019. Share your code and add comments to guide us through it.
2) How many unique researchers you got in 2019?
3) Explain one or two decisions you took during the web-scraping exercise, for 2019 or any other year. Why did you take this choice? How might your decision impact the final number of authors?

In [3]:
#Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.notebook import tqdm
import json

### Scraping the oral presentations
For this session it can be seen from the html that each presentation is marked with the <br> tag, for finding every presentation the ``soup.findall()´´  function can be used.

In [2]:
url = "https://2019.ic2s2.org/oral-presentations/"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

researchers = soup.find_all("br")
researchers = [i.previous_sibling for i in researchers]

print(researchers)
researchers = researchers[9:] #Some tags in the beginning are included which can simply be cut out

[<strong>Rooms per track during main conference</strong>, ' Keynotes', ' Track A', ' Track B (except 1B on Thu)', ' Track C', ' Track D (except 1D & 2D on Thu)', ' Track E', ' Track F', ' – Thursday July 18, 11:00 – 12:30 ', <em>Chair: Ziv Epstein</em>, '11:00 – 11:15 – Jieyu Ding, Qiusi Sun, Jingwen Zhang. Classifying and Understanding the Semantic Structures of Vaccine Misinformation on Twitter', '11:15 – 11:30 – Mahmoudreza Babaei, Juhi Kulshrestha, Abhijnan Chakraborty, Elissa M. Redmiles, Meeyoung Cha, Krishna Gummadi. Analyzing Biases in Perception of Truth in News Stories and Their Implications for Fact Checking', '11:30 – 11:45 – Alexandre Bovet, Hernan Makse. Dynamics and influence of misinformation and traditional fact-based news in Twitter', '11:45 – 12:00 – Ziv Epstein, Mohsen Mosleh, Antonio Arechar, Gordon Pennycook, David Rand. Primed for accuracy: A cognitive approach to fighting misinformation on Twitter', '12:00 – 12:15 – Sunandan Chakraborty, Tarunima Prabhakar, Joyo

Now some formatting have to be doen by splitting on punctuations "." and dashes "-". Also in the last line of the code block we have to go from list of lists to just one list, simutaniously one can remove any whitespaces that may occur in befor each author.

In [3]:
d = [str(i).split('.') for i in researchers]
d = [i[0] for i in d]
d = [i.split(' – ') for i in d]

dt = []
for i in d:
  for j in i:
    if ':' not in j and j != '':
      dt.append(j)

d = [i.split(',') for i in dt]
d = [item[1:] if item[0] == ' ' else item for sublist in d for item in sublist]
d = list(set(d)) # In order to remove duplicates a set of the list can be used

In [4]:
len(d) #Check the length of the author list

477

Now check if it looks right. We see that some titles are in the author data-set, because of this we can make the assumption that if any "author-name" is longer than 4 words we take it as a title.

In [5]:
oral_presenters = d.copy()

In [6]:
name_length = lambda x: len(x.split(' '))

oral_presenters = [name for name in oral_presenters if name_length(name) < 4]

Now check if the list again

### Scraping the poster session
For this html 

In [8]:
url = "https://2019.ic2s2.org/posters/"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

researchers = soup.find_all("li")
researchers = [i.text.strip().replace("\n", "").replace("\t", "") for i in researchers if i.find("span")]

print(researchers)

['Program Program overviewAll keynotes & panelsAll oral presentationsAll poster presentationsAll tutorials', 'Warmup General informationJob FairDatathon', 'Attending / Local info Presenter informationList of delegatesRegistration———–Conference VenueTransportationAccommodationSocial EventsChild care', 'Organizers Conference ChairsProgram CommitteePrevious Editions', 'Frank Peter Pijpers\xa0Hypothesis testing of causality between time series', 'Martin Lukac and André Grow\xa0Recruiting mechanisms on online labour markets: an agent-based model', 'Kiran Garimella, Gianmarco De Francisci Morales, Aristides Gionis and Michael Mathioudakis\xa0Political Discourse on Social Media: Echo Chambers, Gatekeepers, and the Price of Bipartisanship', 'Eaman Jahani, Peter Krafft, Yoshihiko Suhara, Esteban Moro and Alex Pentland\xa0ScamCoins, S*** Posters, and the Search for the Next Bitcoin TM : Collective Sensemaking in Cryptocurrency Discussions', 'Gabriel Murray\xa0Predicting Small Group Interaction D

In [9]:
d = [i.split('\xa0')[0] for i in researchers][4:]
d = [i.split('and') for i in d]
d = [item for sublist in d for item in sublist]
d = [i.split(',') for i in d]
d = [item for sublist in d for item in sublist if item != '']
# One last clean up for empty spaces in the end or beginnig of a word
d = [name[1:] if name[0] == ' ' else name for name in d]
# Do the same for the names ending with white space
d = [name[:-1] if len(name) >1 and name[-1] == ' ' else name for name in d]
d = list(set(d))

In [10]:
#Get the length of the list
len(d)

443

# Part 2: Getting data from the Semantic Scholar API

First load the id's which was saved into json file, and then convert it to a dictionary. Note that if the author could not be found it was simply marked it with not found, so first that will have to be sorted out of the data.

In [2]:
author_database = pd.read_json('/Users/madsandersen/CompSocialScience/02467-CSS/all_authors.json', typ='series').to_dict()

### Get the papper and the the other required entrances
We choose to do a subset due to resource limitations. However the code demonstrates how the querying could be done, given enough time. Note that there is a 100 request limitation per 5 min, which is why a counter is initialized and incremented everytime a request is made to the server. When the limitation is reached the time module is used to make the loop wait for 5 minutes. To be on the safe side the number of requests is set to 95 and 10 seconds are added to the timer pause.

Exercises which needs to be solved
1) Share the number of authors you will use as starting point in this exercises. Add a comment clarifying how many IC2S2 editions you included and if the collaborators were included or not.
2) Share the code you have used to solve the exercise above. (See code cell below)
3) How long is your final Author dataframe? How long is your final Paper dataframe?


### 1)
The data-base of authors which was created, was 112,000 authors in total so this is all the authors from IC2S2 and all of their collaborators in other projects. For this exercise only a subset of size xx were considered, in order to keep the running time reasonable.

### 2) Code for generating data-sets
The following code cells generates the 3 data-sets. Above each there is a short description of what the code does.

In the first part og this, 3 dataframes are generated and the number of authors being quried for is specified in a variable. Note that that for the "Author_dataset", the author id is created as an index, this is just to keep it more manageable and can easily be converted to a column if needed.

In [23]:
n_authors = 10000
Author_dataset = pd.DataFrame(index = list(author_database.keys())[:n_authors],columns = ['name','aliases','citationCount','field'])
Ad_fields = ['name','aliases','citationCount','field']

Paper_dataset = pd.DataFrame(columns = ['paperId','title','year','DOI','citationCount','fields','authorIds'])
Abstract_dataset = pd.DataFrame(columns = ['paperId','abstract'])

The function below is a helper function for creating paper dataset, and also the abstract dataset. It loops through the collection of papers given to it, and distributes the information into the correct dataframes. Beyond this it checks if the paper already exists in the data-set such that replicas are avoided. Some of the fields do need to have an option for a none value, since the api will not return anything if the field does not exist for the author. For the author ids a list is made of all the authors, and the it is joined into a single string. This so that it becomes easier to parse to a dataframe, and since the id's are seperated with a "," it is easy to get a list of them when working with the dataframe using the .split() string method.
Another thing to mention is the fact that a dataframe is created for each row, and then appended to the final dataset.

In [24]:
def write_paper_data(papers,Abstract_dataset,Paper_dataset):
  #print('test')
   #### Paper dataset
  d = {}
  a = {}
  for i,paper in enumerate(papers):
    if paper['paperId'] not in Paper_dataset['paperId']:
      d['paperId'] = paper['paperId']
      d['title'] = paper['title']
      d['year'] = paper['year']
      d['DOI'] = paper['externalIds']['DOI'] if 'DOI' in paper['externalIds'].keys() else None
      d['citationCount'] = paper['citationCount']
      d['fields'] = [r['s2FieldsOfStudy'][0]['category'] if len(r['s2FieldsOfStudy']) != 0 else None for r in papers]
      d['authorIds'] = ','.join([au['authorId'] for au in paper['authors'] if au['authorId'] in author_database.keys()])
      df = pd.DataFrame([d])
      Paper_dataset = pd.concat([Paper_dataset,df])

    # Create the abstract data-set
    if paper['paperId'] not in Abstract_dataset['paperId']:
      a['paperId'] = paper['paperId']
      a['abstract'] = paper['abstract']
      df1 = pd.DataFrame([a])
      Abstract_dataset = pd.concat([Abstract_dataset,df1])

  return Paper_dataset.copy(),Abstract_dataset.copy()
    #df['authorIds'] = df.apply(lambda x: [author['authorId'] for author in x['authorIds']],axis = 1)
    #Paper_dataset = pd.concat([Paper_dataset,df])

The code bellow is where the query takes place, first of the fields are defined ie. what should be quired for. Next thing is to loop through all authors being considered, the query is done using their author id, the returned object is then converted into a json object so that it can be accessed as a dictionary. most of the required fields is just copy pasting them into the dataframe. However for the citation count, each authors paper has to be iterated and the citation count can be done as sum of the list of citation counts. For the most frequent field of study, all the fields of studies the author has been involved in, is made in to a list, and the most frequent element in that list is found. One more thing about the code is that is saves the dataframes to csv every 95 queries, at the same time it sleeps for 1 second in order to avoid exceeding the request limit from the api.

Note:
Our group got an api key for 100 req/sec for the semanthic scholar api. It cant be distributed so either paste in your own api-key or delete the lines with headers and as an argument in the request call. Also this limits the request frequency to 100 per 5 min, so in the time.sleep() it should wait for 5 min (see the last lines of code).

In [ ]:
#How many authors should be considered in the dataset
# Init dataset
data = {id: {} for id in list(author_database.keys())[:n_authors]}

fields = f'name,aliases,papers.title,papers.abstract,papers.year,papers.s2FieldsOfStudy,papers.citationCount,papers.paperId,papers.externalIds,papers.authors'

exception_counter = 0
query_counter = 0
total_queries = 0
for id in tqdm(list(author_database.keys())[:n_authors]):
  if id == 'not found':
    continue

  try:
    my_url = f'https://api.semanticscholar.org/graph/v1/author/{id}?fields={fields}' #BASE_URL + VERSION + RESOURCE
    params = {'authorId': id}
    headers = {"x-api-key": 'e3X5pFPRiu8cWqGesihE3877Nq1Iuf3l88bTUe4Z'}

    r = requests.get(my_url, params=params,headers=headers).json()
    query_counter += 1
    total_queries += 1

    #############################Author_dataset##########################################
    Author_dataset['name'][id] =  r['name']
    Author_dataset['aliases'][id] = ','.join(r['aliases']) if r['aliases'] is not None else 'None'
    Author_dataset['citationCount'][id] = sum([paper['citationCount'] for paper in r['papers']])

    # Make a placeholder, so the most frequent can be determined:
    data[id]['papers.s2FieldsOfStudy'] = [paper['s2FieldsOfStudy'][0]['category'] if len(paper['s2FieldsOfStudy']) != 0 else None for paper in r['papers']]
    freq_field = max(set(data[id]['papers.s2FieldsOfStudy']), key = data[id]['papers.s2FieldsOfStudy'].count)
    Author_dataset['field'][id] = freq_field

    ###########################Paper_dataset ##########################################
    Paper_dataset,Abstract_dataset = write_paper_data(r['papers'],Abstract_dataset,Paper_dataset)

  except:
    print(f"Exception caught for author:{id}")
    exception_counter += 1
    time.sleep(1)


  if query_counter > 95:
    #Save to csv, this is done in each iteration as a back up, makes it a little slower but ensures that data aren't lost if/when kernel times out.
    Author_dataset.to_csv('Author_dataset.csv')
    Paper_dataset.to_csv('Paper_dataset.csv')
    Abstract_dataset.to_csv('Abstract_dataset.csv')

    time.sleep(1)
    query_counter = 0

#save_path = f'data_{total_queries}.json'
#with open(save_path, 'w') as f:
#  json.dump(data, f, indent=4)

  0%|          | 0/10000 [00:00<?, ?it/s]

Exception caught for author:2149106755
Exception caught for author:40025432
Exception caught for author:144935248
Exception caught for author:39647400


KeyboardInterrupt: 

4068 authors

Read the csv file data-set created:

In [13]:
Author_dataset = pd.read_csv('/Users/madsandersen/CompSocialScience/02467-CSS/Author_dataset.csv')
Author_dataset.columns = Author_dataset.columns.str.replace('Unnamed: 0','AuthorID')
Paper_dataset = pd.read_csv('/Users/madsandersen/CompSocialScience/02467-CSS/Paper_dataset.csv')

### 3)
The results below are from quering from 4068 authors in total
###### Resulting lengths:
The length of the final author dataframe is: xx
The length of the final paper dataframe is: xx

In [15]:
pd_length = len(Paper_dataset['paperId'].unique())
au_length = len(Author_dataset['AuthorID'].unique())
print(f'Length of the Paper-dataset: {pd_length}')
print(f'Length of the Auhtor-dataset: {au_length}')

Length of the Paper-dataset: 219246
Length of the Auhtor-dataset: 10000


In [14]:
Author_dataset.head()

,AuthorID,name,aliases,citationCount,field
0,3.228613e+07,Jonathan Zong,None,40.0,Art
1,1.454267e+08,J. N. Matias,"J Nathan Matias,J. Nathan Matias,Jorge Nathan ...",606.0,Computer Science
2,4.622519e+07,Josh Pollock,None,108.0,Computer Science
3,1.666685e+09,Dylan Wootton,Dylan R. Wootton,21.0,Art
4,2.795670e+06,Arvind Satyanarayan,None,2498.0,Computer Science


In [7]:
r #2606408

{'authorId': '2606408',
 'name': 'Douglas Zytko',
 'aliases': ['Doug Zytko'],
 'papers': [{'paperId': '0755a4a766ad6e888297f24a26d3e6f5f0fb7af0',
   'externalIds': {'DBLP': 'conf/cscw/DateyAZ22',
    'DOI': '10.1145/3500868.3559472',
    'CorpusId': 253385713},
   'title': 'Repurposing AI in Dating Apps to Augment Women’s Strategies for Assessing Risk of Harm',
   'abstract': 'In this paper we present emerging findings from an interview study with women in North America about how AI could be designed to prevent online dating-facilitated violence by augmenting their strategies for risk awareness. The study is motivated by gender disparities in harm through online dating, and the relative absence of dating app designs that prioritize women. Findings show that women are receptive to the notion of AI that augments their existing strategies for assessing risk of harm with meeting a particular user face-to-face. They outline various physical and non-physical harms that they attempt to reduce

In [53]:
for idx,id in enumerate(tqdm(list(author_database.keys())[:n_authors])):

  ####Author dataset
  Author_dataset['name'][id]= data[id]['name']
  Author_dataset['aliases'][id]= data[id]['aliases']
  Author_dataset['citationCount'][id]= sum(data[id]['papers.citationCount'])

  #Most frequent field
  field = max(set(data[id]['papers.s2FieldsOfStudy']), key = data[id]['papers.s2FieldsOfStudy'].count)
  Author_dataset['field'][id] = field

  #### Paper dataset
  d = {}
  a = {}
  for i,paper in enumerate(data[id]['papers.paperId']):
    d['paperId'] = paper
    d['title'] = data[id]['papers.title'][i]
    d['year'] = data[id]['papers.year'][i]
    d['DOI'] = data[id]['papers.DOI'][i]
    d['citationCount'] = data[id]['papers.citationCount'][i]
    d['fields'] = data[id]['papers.s2FieldsOfStudy'][i]
    d['authorIds'] = ','.join([a['authorId'] for a in data[id]['papers.Authors'][i] if a['authorId'] in author_database.keys()])
    df = pd.DataFrame([d])
    #df['authorIds'] = df.apply(lambda x: [author['authorId'] for author in x['authorIds']],axis = 1)
    Paper_dataset = pd.concat([Paper_dataset,df])

    # Create the abstract data-set
    if paper not in Abstract_dataset['paperId']:
      a['paperId'] = paper
      a['abstract'] = data[id]['abstract'][i]
      df1 = pd.DataFrame([a])
      Abstract_dataset = pd.concat([Abstract_dataset,df1])

  0%|          | 0/10 [00:00<?, ?it/s]

a83becaa4816161192474f1c7c98aecbc015a374
aba5722bc4b2bd5c694488a431a721fa79c2256d
ac47d785610ab432b59375f4b1c2d97379050132
fde65235dc02a0253daa911adbb10f0a25887ff2
08764a956f31ea736cb95ea01727b1bc52747770
342536c7d1d0369dff1e5e0f1d5876a8dc63042c
ed13b13e317f6fba239711be728dd802d2c9047d
bbb842e20bfb66c966683ace190eb8e42058c18c
db0cd343ceb532ae48d1e0b3e701fd606d5ddfb5
60553c277ed864f0db1181d0da55f943e0fe2bf2
a820eb290d0cbe7bd9751fcb9b1f4eb92fffa947
a83becaa4816161192474f1c7c98aecbc015a374
1d0923487aa3834b7eb9fb6e39248144c1f51f0d
2c7fdd8300e4e7720d8a117210ca26e62fd5c4f3
7d87044328939b67073e4a28a0a063319cc7e1ea
02d954ba9ac2b76b7ae8e8e582ecde521948cfcc
47af5455b2f0ae8f7ae64c683153133a06cd48c2
86f5799ee0e55fbdb6867d5880324cfbf286d20d
879425fc9b8b78956eb6333b6969723e3ca9780e
b00545a2731a3b5c06faee400dcad70472686a88
b58b73e754c5f836890a5c06e2b018ff4ac7f99c
fd4d5263e9b5d474c6f9ace8ecf22254159506f7
bc954e2ec4e10ba2249178e236e3aac99afa11c5
c828be2d531db58d919a1988e0dcd40dc4ac731b
da8c2f7225bd5c1f

In [ ]:
#How many authors should be considered in the dataset
# Init dataset
data = {id: {} for id in list(author_database.keys())[:n_authors]}

fields = f'name,aliases,papers.title,papers.abstract,papers.year,papers.s2FieldsOfStudy,papers.citationCount,papers.paperId,papers.externalIds,papers.authors'

exception_counter = 0
query_counter = 0
total_queries = 0
for id in tqdm(list(author_database.keys())[:n_authors]):
  if id == 'not found':
    continue

  try:
    my_url = f'https://api.semanticscholar.org/graph/v1/author/{id}?fields={fields}' #BASE_URL + VERSION + RESOURCE
    params = {'authorId': id}
    r = requests.get(my_url, params=params).json()
    query_counter += 1
    total_queries += 1

    #Author_dataset
    data[id]['name'] =  r['name']
    data[id]['aliases'] = ','.join(r['aliases']) if r['aliases'] is not None else 'None'
    data[id]['papers.title'] = [paper['title'] for paper in r['papers']]
    data[id]['abstract'] = [paper['abstract'] for paper in r['papers']]
    data[id]['papers.year'] = [paper['year'] for paper in r['papers']]
    data[id]['papers.s2FieldsOfStudy'] = [paper['s2FieldsOfStudy'][0]['category'] if len(paper['s2FieldsOfStudy']) != 0 else None for paper in r['papers']]
    data[id]['papers.citationCount'] = [paper['citationCount'] for paper in r['papers']]

    data[id]['papers.paperId'] = [paper['paperId'] for paper in r['papers']]
    data[id]['papers.DOI'] = [paper['externalIds']['DOI'] if 'DOI' in paper['externalIds'].keys() else 'None' for paper in r['papers']]
    data[id]['papers.Authors'] = [paper['authors'] for paper in r['papers']]

  except:
    print(f"Exception caught for author:{id}")
    exception_counter += 1
    time.sleep(60*5+10)


  if query_counter > 95:
    time.sleep(60*5+10)
    query_counter = 0

save_path = f'data_{total_queries}.json'
with open(save_path, 'w') as f:
  json.dump(data, f, indent=4)